In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [2]:
df_description = pd.read_csv('./data/data_definition.txt', sep='\t')

df_train_genba = pd.read_csv('./data/train_genba.tsv', sep='\t')
df_train_goto = pd.read_csv('./data/train_goto.tsv', sep='\t')

df_train = df_train_goto.merge(df_train_genba, on='pj_no', how='left')
df_train.drop('id', axis=1, inplace=True)

df_test_genba = pd.read_csv('./data/test_genba.tsv', sep='\t')
df_test_goto = pd.read_csv('./data/test_goto.tsv', sep='\t')

df_test = df_test_goto.merge(df_test_genba, on='pj_no', how='left')
df_test.drop('id', axis=1, inplace=True)

## Preprocessing

In [3]:
def fill_city_name(name):
    if '市' not in name and '郡' not in name:
        name = '市' + name
    return name

def split_address(df):
    df['jukyo'] = df['jukyo'].str.slice(start=3).str.replace(r'[ヶｹ]', 'ケ')
    df['jukyo'] = df['jukyo'].apply(fill_city_name)
    city_split = df['jukyo'].str.split(r'[市郡]', n=1, expand=True)
    df['city'] = city_split[0]
    street_split = city_split[1].str.split(r'[町区]', n=1, expand=True)
    df['street'] = street_split[0]
    df['address_detail'] = street_split[1].str.strip().replace('', None)
    return df

df_train = split_address(df_train)
df_test = split_address(df_test)

df_train.drop(['kaoku_um', 'toshikuiki2'], axis=1, inplace=True)
df_test.drop(['kaoku_um', 'toshikuiki2'], axis=1, inplace=True)

In [4]:
def combine(row, combine_list, tup):
    l = set()
    for col in tup:
        if pd.notnull(row[col]):
            l.add(row[col])
    combine_list.append(','.join(l))
    
combine_cols = ['yoto', 'road_hk']
for i, tup in enumerate([['yoto1', 'yoto2'], ['road1_hk', 'road2_hk', 'road3_hk', 'road4_hk']]):
    combine_train = []
    combine_test = []
    
    combine_col_name = combine_cols[i]
    _ = df_train.apply(lambda row: combine(row, combine_train, tup), axis=1)
    _ = df_test.apply(lambda row: combine(row, combine_test, tup), axis=1)

    count_vectorizer = CountVectorizer(min_df=100)
    combine_train_matrix = count_vectorizer.fit_transform(combine_train).todense()
    combine_test_matrix = count_vectorizer.transform(combine_test).todense()
    for i in range(combine_train_matrix.shape[1]):
        df_train['%s_%d' % (combine_col_name, i)] = combine_train_matrix[:, i]
        df_test['%s_%d' % (combine_col_name, i)] = combine_test_matrix[:, i]
    for col in tup:
        df_train.drop(col, axis=1, inplace=True)
        df_test.drop(col, axis=1, inplace=True)

In [5]:
continue_features = list(df_description[df_description['データ型'] == '数値']['項目名'])
objective = 'keiyaku_pr'
categorical_features = list(df_train)

for f in continue_features+[objective]:
    if f in categorical_features:
        categorical_features.remove(f)
        
for col in categorical_features:
    vc = dict(df_train[col].value_counts())
    df_train['%s_freq' % col] = df_train[col].apply(lambda row: vc[row] if row in vc else 0)
    df_test['%s_freq' % col] = df_test[col].apply(lambda row: vc[row] if row in vc else 0)
    continue_features.append('%s_freq' % col)

## Train

In [6]:
for col in categorical_features:
    dt = df_train[col].dtype
    if dt == int or dt == float:
        df_train[col].fillna(-1, inplace=True)
        df_test[col].fillna(-1, inplace=True)
    else:
        df_train[col].fillna('', inplace=True)
        df_test[col].fillna('', inplace=True)
        
for col in categorical_features:
    vc = dict(df_train[col].value_counts())
    for k, v in vc.items():
        if v <= 3:
            df_train[col].replace(k, 'Low_freq', inplace=True)
            df_test[col].replace(k, 'Low_freq', inplace=True)
    le = LabelEncoder()
    df_train[col] = le.fit_transform(df_train[col])
    mapping = {v: i for i, v in enumerate(le.classes_)}
    df_test[col] = df_test[col].apply(lambda x: mapping[x] if x in mapping else -1)

splitter = KFold(n_splits=5, shuffle=True, random_state=28)
prediction_list = []
best_scores = []
for train_idx, valid_idx in splitter.split(df_train):
    train, valid = df_train.iloc[train_idx], df_train.iloc[valid_idx]
    X_train, y_train = train.drop('keiyaku_pr', axis=1), np.log(train['keiyaku_pr']+1)
    X_valid, y_valid = valid.drop('keiyaku_pr', axis=1), np.log(valid['keiyaku_pr']+1)
    regressor = lgb.LGBMRegressor(n_estimators=20000, learning_rate=0.01, silent=False, random_state=28,
                                 bagging_fraction=0.9, bagging_freq=1, feature_fraction=0.8, lambda_l1=0.01)
    regressor.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=500,
                  categorical_feature=categorical_features)
    prediction_list.append(regressor.predict(df_test[df_train.drop(objective, axis=1).columns]))
    best_scores.append(regressor.best_score_['valid_0']['l2'])

print("5-fold cv mean l2 %.8f" % np.mean(best_scores))

df_submission = pd.read_csv('./data/sample_submit.tsv', sep='\t', names=['id', 'pred'])

df_submission['pred'] = np.exp(np.mean(prediction_list, axis=0))-1
df_submission.to_csv('submission.tsv', sep='\t', header=None, index=False)

# 0.01227741

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['address_detail', 'bas_toho1', 'bas_toho2', 'bastei_nm1', 'bastei_nm2', 'bokachiiki', 'bus_yohi', 'chikukeikaku', 'city', 'eki_nm1', 'eki_nm2', 'fi3m_yohi', 'fi4m_yohi', 'gas', 'gesui', 'hiatari', 'hokakisei1', 'hokakisei2', 'hokakisei3', 'hokakisei4', 'hokakyoka', 'hw_status', 'jigata', 'josui', 'jukyo', 'kaihatsukyoka', 'kborjs', 'keikakuroad', 'kinshijiko', 'kobetsu1', 'kobetsu2', 'kobetsu3', 'kobetsu4', 'kodochiku', 'levelplan', 'road1_sb', 'road2_sb', 'road3_sb', 'road4_sb', 'road_hk_0', 'road_hk_1', 'road_hk_2', 'road_hk_3', 'road_st', 'rosen_nm1', 'rosen_nm2', 'rs_e_kdate2', 'rs_e_kdate3', 'rs_e_m_ari', 'rs_e_m_nashi', 'rs_e_parking', 'rs_e_tahata', 'rs_e_zoki', 'rs_n_kdate2', 'rs_n_kdate3', 'rs_n_m_ari', 'rs_n_m_nashi', 'rs_n_parking', 'rs_n_tahata', 'rs_n_zoki', 'rs_s_kdate2', 'rs_s_kdate3', 'rs_s_m_ari', 'rs_s_m_nashi', 'rs_s_pa

[1]	valid_0's l2: 0.0676629
Training until validation scores don't improve for 500 rounds.
[2]	valid_0's l2: 0.0669061
[3]	valid_0's l2: 0.0662154
[4]	valid_0's l2: 0.0654964
[5]	valid_0's l2: 0.0647812
[6]	valid_0's l2: 0.0640938
[7]	valid_0's l2: 0.0634065
[8]	valid_0's l2: 0.0627274
[9]	valid_0's l2: 0.0620498
[10]	valid_0's l2: 0.0614117
[11]	valid_0's l2: 0.0607968
[12]	valid_0's l2: 0.0601915
[13]	valid_0's l2: 0.0595964
[14]	valid_0's l2: 0.0589881
[15]	valid_0's l2: 0.058395
[16]	valid_0's l2: 0.057813
[17]	valid_0's l2: 0.0572856
[18]	valid_0's l2: 0.0567618
[19]	valid_0's l2: 0.0562083
[20]	valid_0's l2: 0.0556511
[21]	valid_0's l2: 0.055113
[22]	valid_0's l2: 0.0545739
[23]	valid_0's l2: 0.0540552
[24]	valid_0's l2: 0.0535372
[25]	valid_0's l2: 0.0530418
[26]	valid_0's l2: 0.0525466
[27]	valid_0's l2: 0.0520899
[28]	valid_0's l2: 0.0516395
[29]	valid_0's l2: 0.0511659
[30]	valid_0's l2: 0.0507153
[31]	valid_0's l2: 0.0502924
[32]	valid_0's l2: 0.0498444
[33]	valid_0's l2: 0.

[277]	valid_0's l2: 0.0211534
[278]	valid_0's l2: 0.0211301
[279]	valid_0's l2: 0.0211094
[280]	valid_0's l2: 0.0210911
[281]	valid_0's l2: 0.0210725
[282]	valid_0's l2: 0.0210493
[283]	valid_0's l2: 0.0210305
[284]	valid_0's l2: 0.0210081
[285]	valid_0's l2: 0.0209876
[286]	valid_0's l2: 0.0209673
[287]	valid_0's l2: 0.0209464
[288]	valid_0's l2: 0.0209231
[289]	valid_0's l2: 0.020902
[290]	valid_0's l2: 0.0208862
[291]	valid_0's l2: 0.0208641
[292]	valid_0's l2: 0.0208418
[293]	valid_0's l2: 0.0208205
[294]	valid_0's l2: 0.0208008
[295]	valid_0's l2: 0.0207877
[296]	valid_0's l2: 0.0207767
[297]	valid_0's l2: 0.0207617
[298]	valid_0's l2: 0.0207488
[299]	valid_0's l2: 0.0207339
[300]	valid_0's l2: 0.0207129
[301]	valid_0's l2: 0.0206975
[302]	valid_0's l2: 0.0206779
[303]	valid_0's l2: 0.0206601
[304]	valid_0's l2: 0.0206423
[305]	valid_0's l2: 0.0206279
[306]	valid_0's l2: 0.0206109
[307]	valid_0's l2: 0.0205921
[308]	valid_0's l2: 0.0205718
[309]	valid_0's l2: 0.0205553
[310]	valid

[557]	valid_0's l2: 0.0182823
[558]	valid_0's l2: 0.0182786
[559]	valid_0's l2: 0.0182742
[560]	valid_0's l2: 0.0182704
[561]	valid_0's l2: 0.0182629
[562]	valid_0's l2: 0.0182551
[563]	valid_0's l2: 0.0182504
[564]	valid_0's l2: 0.0182437
[565]	valid_0's l2: 0.0182422
[566]	valid_0's l2: 0.0182397
[567]	valid_0's l2: 0.0182366
[568]	valid_0's l2: 0.0182315
[569]	valid_0's l2: 0.0182238
[570]	valid_0's l2: 0.0182206
[571]	valid_0's l2: 0.0182162
[572]	valid_0's l2: 0.0182133
[573]	valid_0's l2: 0.0182079
[574]	valid_0's l2: 0.0182019
[575]	valid_0's l2: 0.0181982
[576]	valid_0's l2: 0.0181924
[577]	valid_0's l2: 0.0181894
[578]	valid_0's l2: 0.0181864
[579]	valid_0's l2: 0.0181828
[580]	valid_0's l2: 0.0181819
[581]	valid_0's l2: 0.0181783
[582]	valid_0's l2: 0.0181732
[583]	valid_0's l2: 0.0181693
[584]	valid_0's l2: 0.0181636
[585]	valid_0's l2: 0.0181617
[586]	valid_0's l2: 0.0181611
[587]	valid_0's l2: 0.0181568
[588]	valid_0's l2: 0.0181495
[589]	valid_0's l2: 0.0181455
[590]	vali

[846]	valid_0's l2: 0.0175101
[847]	valid_0's l2: 0.0175086
[848]	valid_0's l2: 0.0175059
[849]	valid_0's l2: 0.0175041
[850]	valid_0's l2: 0.0175024
[851]	valid_0's l2: 0.0175022
[852]	valid_0's l2: 0.0175005
[853]	valid_0's l2: 0.017499
[854]	valid_0's l2: 0.0174993
[855]	valid_0's l2: 0.0174963
[856]	valid_0's l2: 0.017495
[857]	valid_0's l2: 0.0174929
[858]	valid_0's l2: 0.0174939
[859]	valid_0's l2: 0.0174938
[860]	valid_0's l2: 0.0174913
[861]	valid_0's l2: 0.0174878
[862]	valid_0's l2: 0.0174876
[863]	valid_0's l2: 0.0174908
[864]	valid_0's l2: 0.01749
[865]	valid_0's l2: 0.0174877
[866]	valid_0's l2: 0.0174869
[867]	valid_0's l2: 0.0174854
[868]	valid_0's l2: 0.0174849
[869]	valid_0's l2: 0.0174825
[870]	valid_0's l2: 0.0174817
[871]	valid_0's l2: 0.017483
[872]	valid_0's l2: 0.0174826
[873]	valid_0's l2: 0.0174811
[874]	valid_0's l2: 0.0174809
[875]	valid_0's l2: 0.0174797
[876]	valid_0's l2: 0.01748
[877]	valid_0's l2: 0.0174769
[878]	valid_0's l2: 0.0174762
[879]	valid_0's l

...

[1922]	valid_0's l2: 0.0119328
[1923]	valid_0's l2: 0.0119324
[1924]	valid_0's l2: 0.0119317
[1925]	valid_0's l2: 0.0119321
[1926]	valid_0's l2: 0.0119319
[1927]	valid_0's l2: 0.0119324
[1928]	valid_0's l2: 0.0119325
[1929]	valid_0's l2: 0.011934
[1930]	valid_0's l2: 0.0119338
[1931]	valid_0's l2: 0.0119342
[1932]	valid_0's l2: 0.0119344
[1933]	valid_0's l2: 0.0119345
[1934]	valid_0's l2: 0.011933
[1935]	valid_0's l2: 0.0119334
[1936]	valid_0's l2: 0.0119328
[1937]	valid_0's l2: 0.011932
[1938]	valid_0's l2: 0.0119313
[1939]	valid_0's l2: 0.0119306
[1940]	valid_0's l2: 0.011929
[1941]	valid_0's l2: 0.01193
[1942]	valid_0's l2: 0.0119305
[1943]	valid_0's l2: 0.011931
[1944]	valid_0's l2: 0.0119302
[1945]	valid_0's l2: 0.0119302
[1946]	valid_0's l2: 0.0119301
[1947]	valid_0's l2: 0.01193
[1948]	valid_0's l2: 0.011929
[1949]	valid_0's l2: 0.0119288
[1950]	valid_0's l2: 0.0119284
[1951]	valid_0's l2: 0.0119283
[1952]	valid_0's l2: 0.0119283
[1953]	valid_0's l2: 0.0119287
[1954]	valid_0's l

[2195]	valid_0's l2: 0.0119014
[2196]	valid_0's l2: 0.0119007
[2197]	valid_0's l2: 0.0119015
[2198]	valid_0's l2: 0.0119017
[2199]	valid_0's l2: 0.0119013
[2200]	valid_0's l2: 0.0119008
[2201]	valid_0's l2: 0.0119002
[2202]	valid_0's l2: 0.0119001
[2203]	valid_0's l2: 0.0118992
[2204]	valid_0's l2: 0.0118986
[2205]	valid_0's l2: 0.0118986
[2206]	valid_0's l2: 0.0118992
[2207]	valid_0's l2: 0.0119004
[2208]	valid_0's l2: 0.0119007
[2209]	valid_0's l2: 0.0118996
[2210]	valid_0's l2: 0.011899
[2211]	valid_0's l2: 0.0118993
[2212]	valid_0's l2: 0.0118989
[2213]	valid_0's l2: 0.0119
[2214]	valid_0's l2: 0.0118986
[2215]	valid_0's l2: 0.0118976
[2216]	valid_0's l2: 0.0118967
[2217]	valid_0's l2: 0.0118939
[2218]	valid_0's l2: 0.011893
[2219]	valid_0's l2: 0.011892
[2220]	valid_0's l2: 0.0118901
[2221]	valid_0's l2: 0.0118906
[2222]	valid_0's l2: 0.0118903
[2223]	valid_0's l2: 0.0118901
[2224]	valid_0's l2: 0.0118899
[2225]	valid_0's l2: 0.01189
[2226]	valid_0's l2: 0.01189
[2227]	valid_0's l

[2469]	valid_0's l2: 0.0118752
[2470]	valid_0's l2: 0.0118756
[2471]	valid_0's l2: 0.0118748
[2472]	valid_0's l2: 0.0118752
[2473]	valid_0's l2: 0.0118754
[2474]	valid_0's l2: 0.0118756
[2475]	valid_0's l2: 0.0118756
[2476]	valid_0's l2: 0.0118748
[2477]	valid_0's l2: 0.0118749
[2478]	valid_0's l2: 0.0118745
[2479]	valid_0's l2: 0.0118749
[2480]	valid_0's l2: 0.0118741
[2481]	valid_0's l2: 0.0118741
[2482]	valid_0's l2: 0.0118738
[2483]	valid_0's l2: 0.011873
[2484]	valid_0's l2: 0.0118728
[2485]	valid_0's l2: 0.0118732
[2486]	valid_0's l2: 0.0118738
[2487]	valid_0's l2: 0.0118735
[2488]	valid_0's l2: 0.0118722
[2489]	valid_0's l2: 0.0118727
[2490]	valid_0's l2: 0.0118724
[2491]	valid_0's l2: 0.0118732
[2492]	valid_0's l2: 0.0118739
[2493]	valid_0's l2: 0.0118744
[2494]	valid_0's l2: 0.0118748
[2495]	valid_0's l2: 0.011875
[2496]	valid_0's l2: 0.0118752
[2497]	valid_0's l2: 0.0118757
[2498]	valid_0's l2: 0.0118765
[2499]	valid_0's l2: 0.0118761
[2500]	valid_0's l2: 0.0118764
[2501]	val

[2743]	valid_0's l2: 0.0118813
[2744]	valid_0's l2: 0.0118821
[2745]	valid_0's l2: 0.0118816
[2746]	valid_0's l2: 0.0118815
[2747]	valid_0's l2: 0.0118821
[2748]	valid_0's l2: 0.0118821
[2749]	valid_0's l2: 0.0118826
[2750]	valid_0's l2: 0.0118834
[2751]	valid_0's l2: 0.0118843
[2752]	valid_0's l2: 0.0118845
[2753]	valid_0's l2: 0.0118849
[2754]	valid_0's l2: 0.0118853
[2755]	valid_0's l2: 0.0118851
[2756]	valid_0's l2: 0.0118859
[2757]	valid_0's l2: 0.0118853
[2758]	valid_0's l2: 0.0118851
[2759]	valid_0's l2: 0.011886
[2760]	valid_0's l2: 0.0118857
[2761]	valid_0's l2: 0.0118852
[2762]	valid_0's l2: 0.0118857
[2763]	valid_0's l2: 0.0118859
[2764]	valid_0's l2: 0.0118857
[2765]	valid_0's l2: 0.0118856
[2766]	valid_0's l2: 0.0118869
[2767]	valid_0's l2: 0.0118864
[2768]	valid_0's l2: 0.0118867
[2769]	valid_0's l2: 0.0118867
[2770]	valid_0's l2: 0.0118868
[2771]	valid_0's l2: 0.0118858
[2772]	valid_0's l2: 0.0118857
[2773]	valid_0's l2: 0.0118853
[2774]	valid_0's l2: 0.0118853
[2775]	va

[3017]	valid_0's l2: 0.0118886
[3018]	valid_0's l2: 0.0118886
[3019]	valid_0's l2: 0.0118876
[3020]	valid_0's l2: 0.0118877
[3021]	valid_0's l2: 0.0118874
[3022]	valid_0's l2: 0.0118877
[3023]	valid_0's l2: 0.0118875
[3024]	valid_0's l2: 0.0118872
[3025]	valid_0's l2: 0.011887
[3026]	valid_0's l2: 0.0118871
[3027]	valid_0's l2: 0.0118877
[3028]	valid_0's l2: 0.0118874
[3029]	valid_0's l2: 0.0118862
[3030]	valid_0's l2: 0.0118861
[3031]	valid_0's l2: 0.0118861
[3032]	valid_0's l2: 0.0118868
[3033]	valid_0's l2: 0.011887
[3034]	valid_0's l2: 0.0118867
[3035]	valid_0's l2: 0.0118874
[3036]	valid_0's l2: 0.0118867
[3037]	valid_0's l2: 0.0118864
[3038]	valid_0's l2: 0.0118856
[3039]	valid_0's l2: 0.0118859
[3040]	valid_0's l2: 0.0118865
[3041]	valid_0's l2: 0.0118866
[3042]	valid_0's l2: 0.0118867
[3043]	valid_0's l2: 0.0118861
[3044]	valid_0's l2: 0.0118862
[3045]	valid_0's l2: 0.011886
[3046]	valid_0's l2: 0.0118863
[3047]	valid_0's l2: 0.011886
[3048]	valid_0's l2: 0.0118863
[3049]	valid